In [1]:
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script
from io import BytesIO

In [2]:
from ecc import PrivateKey, S256Point, Signature
from helper import decode_base58, SIGHASH_ALL, h160_to_p2pkh_address, hash160, h160_to_p2sh_address
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, TxFetcher
from urllib.request import Request, urlopen
import json

In [3]:
privkey_1 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","big"))
my_address_1 = privkey_1.point.address(testnet=True)
print(my_address_1)

mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8


In [4]:
privkey_2 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","little"))
my_address_2 = privkey_2.point.address(testnet=True)
print(my_address_2)

mwuCRN8bMLWN9ETs29U8KseDkau72M9byF


# Let's try to create a multisignature address that needs both signatures from privkey_1 and privkey_2

In [5]:
m=2
op_m = 82
n=2
op_n=82
length_pubkey=33
pubkey1=privkey_1.point.sec()
pubkey2=privkey_2.point.sec()

In [6]:
pubkey1.hex()

'03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4'

In [7]:
pubkey2.hex()

'031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee'

In [8]:
#redeem_script = Script([op_m, length_pubkey, pubkey1, length_pubkey, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script = Script([op_m, pubkey1, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script

OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

In [9]:
serialized_redeem = redeem_script.serialize()
print(serialized_redeem.hex())
serialized_redeem_raw=serialized_redeem[1:]
print(serialized_redeem_raw.hex())

47522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae
522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae


In [10]:
multi_sig_addr = h160_to_p2sh_address(hash160(serialized_redeem_raw), testnet=True)
multi_sig_addr

'2NAqp3V17G4iBekEwWuYkgzoEyWiULhYZLs'

# Now let's try to send some BTCs to this address

In [17]:
def get_index(outs_list, address):
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(address) or out.script_pubkey.cmds[1] == decode_base58(address):
            return index
    return "output index not found"


def get_amount_utxo(outs_list, index):
    return outs_list[index].amount


def get_tx_ins_utxo(prev_tx_id_list, receiving_address, testnet=True):
    
    tx_ins = []
    
    for prev_tx_id in prev_tx_id_list:
        prev_tx = TxFetcher.fetch(prev_tx_id, testnet)
        print(prev_tx)
        prev_index = get_index(prev_tx.tx_outs, receiving_address)
        tx_in = TxIn(bytes.fromhex(prev_tx_id),prev_index)
        utxo = get_amount_utxo(prev_tx.tx_outs, prev_index)
        print(f"index 1: {prev_index}, amount: {utxo}")
        tx_ins.append({"tx_in": tx_in, "utxo": utxo})
        
    return tx_ins

def calculate_fee(version, tx_ins, tx_outs, locktime, testnet=True, privkey):
    tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
    print(tx)

    # sign the inputs in the transaction object using the private key
    for tx_input in range(len(tx_ins)):
        print(tx.sign_input(tx_input, privkey))
    # print the transaction's serialization in hex

    #Let's calculate the fee and the change:
    tx_size = len(my_tx.serialize().hex())
    fee_per_byte = 8 # I changed from 2 to 10 after sending this transaction because it had really low appeal to miners.
    fee = tx_size * fee_per_byte
    print(f"fee: {fee}")
    return fee

def calculate_change(utxo_list, fee, amountTx):
    total_utxo = sum(utxo_list)
    total_out = sum(amountTx)
    change = total_utxo - fee - total_out
    print(f"change {change}")
    if change > 0:
        return "Not enough utxos"
    #Let's make sure that we are actually spending the exact amount of the UTXO
    total_send=fee+amountTx+change
    diff = total_utxo-total_send
    print(f"total {total_send}, diff: {diff}")
    if diff == 0
        return change
    else:
        return "Not able to calculate the right change"

    

In [21]:
from script import p2sh_script
# Let's build the tx:
tx_outs =[]
tx_ins =[]

change = int(0.01 * 100000000)# we are going to fix this later
amountTx= int(0.01 * 100000000)

#It will send 0.01 to the multisig address
output = TxOut(amountTx, p2sh_script(decode_base58(multi_sig_addr)))
print(output)
tx_outs.append(output)

change_output = TxOut(change, p2pkh_script(decode_base58(my_address_1)))
print(change_output)
tx_outs.append(change_output)

#block explorer: https://testnet.smartbit.com.au/address/mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8/unspent

prev_tx_id_list = ["2f9c289966440d088978f9c674a0e596d47db5a91d6d5fc4df7ed38aee0a2d18",
                  "67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5"]

tx_ins_utxo = get_tx_ins_utxo(prev_tx_id_list, my_address_1, testnet=True)
    
tx_ins = [x["tx_in"] for x in tx_ins_utxo]
print(tx_ins)

utxos = [x["utxo"] for x in tx_ins_utxo]
print(utxos)

my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_tx)

fee = calculate_fee(1, tx_ins, tx_outs, 0, testnet=True, privkey_1)
change = calculate_change(utxos, fee, [amountTx])

#Let's modify the amount in the change_output:
my_tx.tx_outs[-1].amount = change

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_tx.sign_input(tx_input, privkey_1))

print(my_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))

1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 2f9c289966440d088978f9c674a0e596d47db5a91d6d5fc4df7ed38aee0a2d18
version: 1
tx_ins:
9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a:0
tx_outs:
980544:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 0, amount: 980544
tx: 67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:1
tx_outs:
1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
591336:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 1, amount: 591336
[2f9c289966440d088978f9c674a0e596d47db5a91d6d5fc4df7ed38aee0a2d18:0, 67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5:1]
[980544, 591336]
tx: 1125840de36407c691

In [12]:
my_tx.verify_input(0)

OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG

combined script[b'0D\x02 +)\xa5C\xee\xc3\xad\xa1\xb7y7\x0c\x02\xfa\x9eE\x84\x9d\xc6yZ\xb6\xbd\x7f)\xfd\xc51\xc1d\xdcb\x02 1>\xfbs\xd68\xe9\x08\xd3\xe35\xf3\xad\xb2\xb6\xe1\xb4f\x16)\xce>\x1b\xb3\xd5\xb8\xe5m\xbe\xb6\x98I\x01', b"\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4", 118, 169, b'R\x90>\xfc\x10\x04\xde\x01\x88;\xa3h{\xe2\xa8\xeaOk\x1b\x19', 136, 172]



True

In [13]:
li = ["a","b","c"]
print(*li)

a b c


# Now let's try to spend the UTXO from the multi-sig address

In [14]:
reload(tx)

<module 'tx' from '/Users/oscareduardosernarosero/Documents/US/Rice U/blockchain/bitcoin/basics/tx.py'>

In [15]:
###### tx_outs =[]
tx_ins =[]
tx_outs =[]

amountTx= int(0.01 * 100000000)# we'll fix this later after calculating the fee

#It will send back all the money to our address_1 minus the fee
output = TxOut(amountTx,  p2pkh_script(decode_base58(my_address_1)))
print(output)
tx_outs.append(output)

prev1_tx_id = "9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a"
prev_tx1 = TxFetcher.fetch(prev1_tx_id, testnet=True)
print(prev_tx1)

def get_index(outs_list, address):
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(address) or out.script_pubkey.cmds[1] == decode_base58(address):
            return index
    return "output index not found"
    
prev1_index= get_index(prev_tx1.tx_outs, multi_sig_addr)
#prev1_index=0
print(f"index 1: {prev1_index}")

tx_in1 = TxIn(bytes.fromhex(prev1_tx_id),prev1_index)
print(tx_in1)
tx_ins.append(tx_in1)

my_multisig_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_multisig_tx)

# sign the inputs in the transaction object using the private key
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier
# print the transaction's serialization in hex

#Let's calculate the fee and the change:
tx_size = len(my_multisig_tx.serialize().hex())
fee_per_byte = 32 # I changed from 10 to 25 after sending this transaction because apparently it is more expensive for multisig tx.
fee = tx_size * fee_per_byte 
print(f"fee: {fee}")

amountTx = prev_tx1.tx_outs[prev1_index].amount - fee 
print(f"amountTx : {amountTx}")
#Let's make sure that we are actually spending the exact amount of the UTXO
total_send=fee+amountTx
diff = prev_tx1.tx_outs[prev1_index].amount-total_send
print(f"total {total_send}, diff: {diff}")

#Let's modify the amount in the change_output:
my_multisig_tx.tx_outs[0].amount = amountTx

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier

print(my_multisig_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))

1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a
version: 1
tx_ins:
ee337430b19e720f5b1bc4582df972bec0750fcfcbf6143d097121d0e11af478:1
tx_outs:
1000000:OP_HASH160 1060f2fd6058cd100fe4e485455badf453bd63e1 OP_EQUAL
753950:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 0
9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a:0
tx: a716747d4c63bcc054ab55d2baa20ae4d50ff74a0ec31f6d7a6efc48b5438547
version: 1
tx_ins:
9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a:0
tx_outs:
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
redeem_script present
script: OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG
sign_input_multisig commands:

IndexError: index out of range

In [ ]:
my_multisig_tx.verify()

In [ ]:
my_multisig_tx

In [ ]:
rebuilt_script = Script.parse(BytesIO(bytes.fromhex("47522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae")))
rebuilt_script

In [ ]:
for cmd in my_multisig_tx.tx_ins[0].script_sig.cmds:
    try: print(cmd.hex())
    except: print(cmd)

In [ ]:
#my_multisig_tx.tx_outs.pop(0)

In [ ]:
#upsidedown = Script(my_multisig_tx.tx_ins[0].script_sig.cmds[::-1])
straight = Script(my_multisig_tx.tx_ins[0].script_sig.cmds)
rebuilt_script2 = Script(rebuilt_script.cmds[1:])
#upsidedown.cmds.pop(0)
straight.cmds.pop(-1)
print(straight)

In [ ]:
combined_part2 = straight+ rebuilt_script

In [ ]:
from helper import hash256
raw_modified=bytes.fromhex("01000000011af3ab90a1c274ecb1f5e6e499ed2d0b5b2ebc6bbcbab751836ea70c90656e9f0000000047522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52aeffffffff0140f60e00000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac0000000001000000")
raw_script_sig="dc00483045022100e9a2b73ff95e3395034e910d5d69dd3e18e12a28e9dacfd5f3b2894afd5fbf3f022035b50d48506932020fcea1781a0c58e9011918bd2a9cbbb5693027035ca9502b01483045022100f7621bb014d53d8a97c0b5605317dd4a92b53e03dd22e71b1db85c12e92218ff02204b77ff2a5bc9f7921d8a17d36d04ca29cbe25b2061f7c2f5e227e449ba614ee5014847522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae"
z=int.from_bytes(hash256(raw_modified),"big")


In [ ]:
combined_part2.evaluate(z)

In [ ]:
combined_part2

In [ ]:

sec1=bytes.fromhex(str(combined_part2.cmds[4].hex()))
sec2=bytes.fromhex(str(combined_part2.cmds[5].hex()))
der1=bytes.fromhex(str(combined_part2.cmds[1][:-1].hex()))
der2=bytes.fromhex(str(combined_part2.cmds[2][:-1].hex()))
point1=S256Point.parse(sec1)
point2=S256Point.parse(sec2)
sig1=Signature.parse(der1)
sig2=Signature.parse(der2)
print(point1.verify(z,sig1))
print(point2.verify(z,sig2))

## Everything looks right, but the tx is not being mined or is being rejected by nodes with error code 26, message 64.( ERROR: 64: NON-MANDATORY-SCRIPT-VERIFY-FLAG (EXTRA ITEMS LEFT ON STACK AFTER EXECUTION)) Therefore we'll try to grab another input to discard this particular input as the source of the issue. 

### Relevant links:
https://www.reddit.com/r/Bitcoin/comments/7l5e38/serious_a_solution_to_a_p2sh_puzzle_not_accepted/drl22i4/

In [ ]:
###### tx_outs =[]
tx_ins =[]
tx_outs =[]

amountTx= int(0.01 * 100000000)# we'll fix this later after calculating the fee

#It will send back all the money to our address_1 minus the fee
output = TxOut(amountTx,  p2pkh_script(decode_base58(my_address_1)))
print(output)
tx_outs.append(output)


prev1_tx_id = "9a84c7d5b2c67d68742f259f37ebc051f4ad6fe0796e3208abdebc04e77768e9"
prev_tx1 = TxFetcher.fetch(prev1_tx_id, testnet=True)
print(prev_tx1)


    
prev1_index= get_index(prev_tx1.tx_outs, multi_sig_addr)
#prev1_index=0
print(f"index 1: {prev1_index}")

tx_in1 = TxIn(bytes.fromhex(prev1_tx_id),prev1_index)
print(tx_in1)
tx_ins.append(tx_in1)

my_multisig_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_multisig_tx)



# sign the inputs in the transaction object using the private key
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier
# print the transaction's serialization in hex

#Let's calculate the fee and the change:
tx_size = len(my_multisig_tx.serialize().hex())
fee_per_byte = 32 # I changed from 10 to 25 after sending this transaction because apparently it is more expensive for multisig tx.
fee = tx_size * fee_per_byte 
print(f"fee: {fee}")

amountTx = prev_tx1.tx_outs[prev1_index].amount - fee 
print(f"amountTx : {amountTx}")
#Let's make sure that we are actually spending the exact amount of the UTXO
total_send=fee+amountTx
diff = prev_tx1.tx_outs[prev1_index].amount-total_send
print(f"total {total_send}, diff: {diff}")


#Let's modify the amount in the change_output:
my_multisig_tx.tx_outs[0].amount = amountTx

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier


print(my_multisig_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))

In [ ]:
rebuilt_script_sig = Script.parse(BytesIO(bytes.fromhex("dc00483045022100e9a2b73ff95e3395034e910d5d69dd3e18e12a28e9dacfd5f3b2894afd5fbf3f022035b50d48506932020fcea1781a0c58e9011918bd2a9cbbb5693027035ca9502b01483045022100f7621bb014d53d8a97c0b5605317dd4a92b53e03dd22e71b1db85c12e92218ff02204b77ff2a5bc9f7921d8a17d36d04ca29cbe25b2061f7c2f5e227e449ba614ee5014847522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae")))
rebuilt_script_sig                                       

In [24]:
"0100000001e96877e704bcdeab08326e79e06fadf451c0eb379f252f74687dc6b2d5c7849a01000000db00483045022100a626c7a9fa1f920c2b16e3dd1f632be29a187286f6fc4d8280ee8001a9c374af0220036cdbcc194a2e6d2253df17b166421c1b5de2d8948b4700530e513f226ca60701473044022029952ddebb6a2a89d5152a1a5f3cb790ceaced841bb360c1ad420acaf79b7b2b02207f1637c4ca87779fe358c5d17609a75d64a34a7b7a72710fe2ad1f40fe037931014847522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52aeffffffff0180f60e00000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac00000000"

'0100000001e96877e704bcdeab08326e79e06fadf451c0eb379f252f74687dc6b2d5c7849a01000000db00483045022100a626c7a9fa1f920c2b16e3dd1f632be29a187286f6fc4d8280ee8001a9c374af0220036cdbcc194a2e6d2253df17b166421c1b5de2d8948b4700530e513f226ca60701473044022029952ddebb6a2a89d5152a1a5f3cb790ceaced841bb360c1ad420acaf79b7b2b02207f1637c4ca87779fe358c5d17609a75d64a34a7b7a72710fe2ad1f40fe037931014847522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52aeffffffff0180f60e00000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac00000000'

In [25]:
Jupyter.keyboard_manager.enable()

NameError: name 'Jupyter' is not defined